In [1]:
from itertools import count
from IPython.display import clear_output
from pathlib import Path
from time import time, sleep
from line_profiler import LineProfiler
import pickle

In [2]:
%load_ext line_profiler

In [3]:
PATH = Path('./data/last_number.txt')

In [4]:
PATH_CHECKPOINT = Path('./data/Checkpoint.pkl')

## Version 1

In [3]:
def print_digits(number):
    while number > 0:
        digit  = number % 10
        number = number // 10
        print(digit)

In [5]:
# Sadly, this approach of going backwards in the digits to check further is too slow.
# Mostly (probably) because we need to allocate and track a bunch of additional data.
# And we can't just check two or three digits, since that happens way too often.
def soft_check_goliath(number):
    interval = goliath_amount // 2
    divisor  = 10**interval
    original_number = number
    prev_was_six    = False
    prev_location   = -interval
    while number > 0:
        digit  = number % 10
        number = number // divisor
        if digit == 6:
            if prev_was_six:
                return True
                # candidate found, check more intensely
                tmp_number = original_number // 10**(prev_location + 1)
                digit      = tmp_number % 10
                if digit == 6:
                    tmp_number = tmp_number // 10
                    digit      = tmp_number % 10
                    if digit == 6:
                        tmp_number = tmp_number // 10
                        digit      = tmp_number % 10
                        if digit == 6:
                            tmp_number = tmp_number // 10
                            digit      = tmp_number % 10
                            if digit == 6:
                                # we now know of six sixes, which we take as enough evidence to trigger an even more expensive check.
                                return True
            prev_was_six = True
        else:
            prev_was_six = False
        prev_location += interval
    return False

In [35]:
profiler = LineProfiler()
@profiler
def count_sixes_naive(number, divisor):
    six_counter     = 0
    six_counter_max = 0
    while number > 0:
        digit    = number % 10
        number //= divisor
        if digit == 6:
            six_counter    += 1
            six_counter_max = max(six_counter_max, six_counter)
        else:
            six_counter = 0
    return six_counter_max

https://stackoverflow.com/questions/1686004/fastest-way-to-convert-binary-to-decimal

In [6]:
LastNumber = int(PATH.read_text()); LastNumber

570123

In [7]:
goliath = 666*'666'
goliath_amount = len(goliath)

In [8]:
goliath_amount

1998

These divisors are let us only check if ever nth digit is a 6, where n is the exponent $10^n$. The threshold is how many 6s we would expect to see if the number we're checking were to contain a goliath amount of 6s in a row.  

In [54]:
# 1995 / 3 = 665
fat_divisor = 10**665
candidate_treshold = 3

In [20]:
# 1995 / 5 = 399 <-------------- This makes about 0.33% of numbers trigger an expensive check
fat_divisor = 10**399
candidate_treshold = 5

In [34]:
# 1995 / 7 = 285
fat_divisor = 10**285
candidate_treshold = 7

In [37]:
i = LastNumber
Number = 1<<i
tStart = time()
tExpensiveDuration = 0
ExpensiveChecks    = 0
while True:
    t1 = time()
    sixes = count_sixes(Number, fat_divisor)
    t2 = time()
    if sixes >= candidate_treshold:
        ExpensiveChecks += 1
        t3 = time()
        if goliath in str(Number):
            print(f'Found one: {i}')
            break
        t4 = time()
    Number = Number<<1
    i += 1
    
    tEnd = time()
    clear_output(wait=True)
    print(i)
    print(f'#Expensive checks : {ExpensiveChecks} ({round(100* ExpensiveChecks / (i - LastNumber), 2)}%)')
    print(f'Total Duration    : {round(tEnd - tStart, 4)} seconds')
    print(f'Cheap Duration    : {round(t2 - t1, 4)} seconds')
    print(f'Expensive Duration: {round(t4 - t3, 4)} seconds')
    # if (i - LastNumber) > 100: break
    tStart = tEnd

570124
#Expensive checks : 123 (0.36%)
Total Duration    : 0.2803 seconds
Cheap Duration    : 0.2793 seconds
Expensive Duration: 0.4039 seconds


KeyboardInterrupt: 

In [1]:
i

570124

In [25]:
PATH.write_text(f'{i-1}');

In [13]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 21.1059 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_2900\4117333934.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(number, divisor):
     4       101        696.0      6.9      0.0      six_counter     = 0
     5       101        329.0      3.3      0.0      six_counter_max = 0
     6     38279     111773.0      2.9      0.1      while number > 0:
     7     38178    9941772.0    260.4      4.7          digit  = number % 10
     8     38178  200726980.0   5257.7     95.1          number = number // divisor
     9     38178     128177.0      3.4      0.1          if digit == 6:
    10      3793      15622.0      4.1      0.0              six_counter    += 1
    11      3793      43681.0     11.5      0.0              six_counter_max = max(six_counter_max, six_counter)
   

## Version 2

 The order of operations is important here in line 9 and 10. We want the first instruction to be the cache access, so the try triggers immediately should we need to go the slow path. At the same time, we need the last bit of N to be shifted into R, and not Q. Otherwise we could overwrite N directly in line 9. This way we need to make a temporary variable, and then write the result into N in line 13 or 15.

We can probably use the remainder to quickly compute additional digits. Since the remainder represents the last few digits of the number anyway, it basically contains all the information we need, but is smaller and thus faster to work with. This could significantly reduce the number of candidates we generate! We can even make this a parameter to change later. So after ever division, we check if the remainder has a 6 at the last digit, and if it does, check the next k digits as well as long as they continue to contain sixes. We could even use the same condition as before, by simply only increasing the counter if all k digits are sixes.

As long as we get cache hits, we actually don't need to compute or store N (or Q as it's sometimes called) at all, since we only ever require the last bit of the previous computation to continue with the next one. Only R is needed for the decision, and that's a fixed size.    
We (probably) do need the entire number in case of a cache miss, since we then need to calculate the actual new values. Although, the cache miss will always happen at the end of the iteration, and should happen in predictable intervals.  
The new number to add to the cache will also always be relatively small. If we add the new number in iteration j, then by definition in iteration j-1 the division result N would have had to have been zero. Then in iteration j, the remainder has grown enough to be larger than D, causing a 1 to be shifted in from the right.


In [5]:
# profiler = LineProfiler()
# @profiler
def count_sixes(D, Cache, k=1):
    SixCounter    = 0
    SixCounterMax = 0
    N = 0
    J = len(Cache)
    for j in range(J):
        R = (Cache[j] << 1) | N
        if R >= D:
            R = R - D
            N = 1
        else:
            N = 0
        Cache[j] = R
        
        Digit = R % 10
        if Digit == 6:
            SixCounter += 1
            if SixCounter == 5:
                R = (Cache[j-1]) // 10
                for _ in range(k):
                    Digit = R % 10
                    if Digit == 6:
                        R = R // 10
                    else:
                        SixCounter = 1
                        break
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
    if N > 0:  # assert N == 1
        Cache.append(1)
    return SixCounterMax

### Save and Load State

In [6]:
LastNumber = int(PATH.read_text()); LastNumber

4805000

In [30]:
class checkpoint:
    def __init__(self, LastNumber, Cache):
        self.LastNumber = LastNumber
        self.Cache = Cache

def SaveState(i, Cache):
    PATH.write_text(f'{i}')
    Checkpoint = checkpoint(i, Cache)
    with PATH_CHECKPOINT.open(mode='wb') as f:
        pickle.dump(Checkpoint, f)

def LoadState():
    with PATH_CHECKPOINT.open(mode='rb') as f:
        Checkpoint = pickle.load(f)
    return Checkpoint

In [34]:
Checkpoint = LoadState()

In [35]:
LastNumber = Checkpoint.LastNumber
Cache      = Checkpoint.Cache
Divisor    = 10**399
i          = LastNumber
Number     = 1<<i
LastNumber

4810000

### Rebuild cache

In [17]:
def RebuildCache(LastNumber):
    """If the cache gets corrupted, use this to rebuild it.
    Be warned, this can take multiple seconds!"""
    i = LastNumber
    N = 1<<(i - 1)
    D = 10**399
    Cache = []
    while N > 0:
        R = N  % D
        N = N // D
        Cache.append(R)
    return i, D, Cache

In [15]:
%%time
if 0:
    i, Divisor, Cache = RebuildCache(LastNumber)

CPU times: total: 36.6 s
Wall time: 36.6 s


In [16]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 36.5959 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_11084\2438383779.py
Function: RebuildCache at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def RebuildCache(LastNumber):
     4                                               """If the cache gets corrupted, use this to rebuild it.
     5                                               Be warned, this can take multiple seconds!"""
     6         1          6.0      6.0      0.0      i = LastNumber
     7         1       1389.0   1389.0      0.0      N = 1<<(i - 1)
     8         1         34.0     34.0      0.0      D = 10**399
     9         1          6.0      6.0      0.0      Cache = []
    10      3627      16641.0      4.6      0.0      while N > 0:
    11      3626  182975092.0  50462.0     50.0          R = N  % D
    12      3626  182924982.0  50448.1     

### Main

In [21]:
def VerifyCache(i, Divisor, Cache):
    'If all of these checks pass, then it should be fine to use these values to continue the search.'
    Number = 1<<(i - 1)
    assert Divisor == 10**399
    assert Number % Divisor == Cache[0]
    j            = len(Cache) - 1
    j0thDivision = Number // (10**(399*(j  )))
    j1thDivision = Number // (10**(399*(j+1)))
    assert j0thDivision != 0
    assert j1thDivision == 0
    assert j0thDivision % Divisor == Cache[j]
    assert Cache[-1] != 0
def ExpensiveVerifyCache(LastNumber, Cache):
    _, _, Cache2 = RebuildCache(LastNumber)
    assert Cache == Cache2

In [36]:
VerifyCache(i, Divisor, Cache)

In [22]:
%%time
if 0:
    ExpensiveVerifyCache(LastNumber, Cache)

CPU times: total: 36.4 s
Wall time: 36.4 s


In [37]:
len(Cache)

3629

In [38]:
goliath = 666*'666'
goliath_amount = len(goliath); goliath_amount

1998

In [39]:
# 1995 / 5 = 399 <-------------- This makes about 0.33% of numbers trigger an expensive check
# Exponent    = 399
# fat_divisor = 10**Exponent
candidate_treshold = 5

In [41]:
## WARNING!!!
## This cell can NOT be restarted after a keyboard interrupt due to likely inconsistent cache!
t0 = time()
tt = 1 / 60
CandidatesFound = 0
while True:
    for _ in range(5_000):
        t1 = time()
        sixes = count_sixes(Divisor, Cache, k=66)
        t2 = time()
        if sixes >= candidate_treshold:
            CandidatesFound += 1
            Path(f'./candidates/{i}').touch() # do the expensive check offline
        i += 1
        clear_output(wait=True)
        print(i)
        print(f'#Candidates found : {CandidatesFound} ({round(100* CandidatesFound / (i - LastNumber), 4)}%)')
        print(f'Cheap Duration    : {round(t2 - t1, 4)} seconds')
        print(f'Total Duration    : {round(tt * (t1 - t0), 4)} minutes')
    
    VerifyCache(i, Divisor, Cache)
    SaveState(i, Cache)
    print('Checkpoint saved to disk!')
    sleep(6)
    break

4815000
#Candidates found : 0 (0.0%)
Cheap Duration    : 0.001 seconds
Total Duration    : 0.1909 minutes
Checkpoint saved to disk!


In [40]:
i

4810000

In [28]:
VerifyCache(i, Divisor, Cache)

In [31]:
SaveState(i, Cache)

### Profiling

In [43]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 71.5721 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_3740\811362255.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache, k=1):
     4       500       3752.0      7.5      0.0      SixCounter    = 0
     5       500       1724.0      3.4      0.0      SixCounterMax = 0
     6       500       1216.0      2.4      0.0      j = 0
     7   1355000    3627650.0      2.7      0.5      while N > 0:
     8   1354500    3135707.0      2.3      0.4          try:
     9   1354500   17847447.0     13.2      2.5              Q, R = Cache[j]
    10   1354500    6781471.0      5.0      0.9              R    = (R << 1) | (N & 1)
    11   1354500    3770885.0      2.8      0.5              if R >= D:
    12    677462    2444918.0      3.6      0.3                  R = R - D
    13  

In [37]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 80.4808 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_3740\811362255.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache, k=1):
     4       500       4088.0      8.2      0.0      SixCounter    = 0
     5       500       1762.0      3.5      0.0      SixCounterMax = 0
     6       500       1806.0      3.6      0.0      j = 0
     7   1355000    3978183.0      2.9      0.5      while N > 0:
     8   1354500    3367363.0      2.5      0.4          try:
     9   1354500   20750070.0     15.3      2.6              Q, R = Cache[j]
    10   1354500    7626254.0      5.6      0.9              R    = (R << 1) | (N & 1)
    11   1354500    3990719.0      2.9      0.5              if R >= D:
    12    677462    2702527.0      4.0      0.3                  R = R - D
    13  

In [31]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 82.1128 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_3740\811362255.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache, k=1):
     4       500       4156.0      8.3      0.0      SixCounter    = 0
     5       500       1707.0      3.4      0.0      SixCounterMax = 0
     6       500       1381.0      2.8      0.0      j = 0
     7   1355000    3926936.0      2.9      0.5      while N > 0:
     8   1354500    3460942.0      2.6      0.4          try:
     9   1354500   21611697.0     16.0      2.6              Q, R = Cache[j]
    10   1354500    7713463.0      5.7      0.9              R    = (R << 1) | (N & 1)
    11   1354500    3939145.0      2.9      0.5              if R >= D:
    12    677462    2734357.0      4.0      0.3                  R = R - D
    13  

In [52]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 285.764 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_2860\3149308276.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      5000      32379.0      6.5      0.0      SixCounter    = 0
     5      5000      14993.0      3.0      0.0      SixCounterMax = 0
     6      5000      14695.0      2.9      0.0      j = 0
     7   8429120   22667472.0      2.7      0.8      while N > 0:
     8   8424120   20222604.0      2.4      0.7          try:
     9   8424120   79352010.0      9.4      2.8              Q, R = Cache[j]
    10   8424116   41619713.0      4.9      1.5              R    = (R << 1) | (N & 1)
    11   8424116   22725421.0      2.7      0.8              if R >= D:
    12   4206433   15305622.0      3.6      0.5                  R = R - D
    13   420

In [64]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 290.674 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_2860\454176833.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      5000      33996.0      6.8      0.0      SixCounter    = 0
     5      5000      16603.0      3.3      0.0      SixCounterMax = 0
     6      5000      14465.0      2.9      0.0      j = 0
     7   8429120   22883187.0      2.7      0.8      while N > 0:
     8   8424120   20623957.0      2.4      0.7          try:
     9   8424120   18750014.0      2.2      0.6              M    = N
    10   8424120   39660028.0      4.7      1.4              N, R = Cache[j]
    11   8424116   41892683.0      5.0      1.4              R    = (R << 1) | (M & 1)
    12   8424116   23332859.0      2.8      0.8              if R >= D:
    13   4206433  

In [76]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 287.01 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_2860\896728211.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      5000      36689.0      7.3      0.0      SixCounter    = 0
     5      5000      15944.0      3.2      0.0      SixCounterMax = 0
     6      5000      14274.0      2.9      0.0      j = 0
     7   8429120   22576043.0      2.7      0.8      while N > 0:
     8   8424120   19896093.0      2.4      0.7          try:
     9   8424120   25424230.0      3.0      0.9              B    = (N & 1)
    10   8424120   38899041.0      4.6      1.4              N, R = Cache[j]
    11   8424116   35478098.0      4.2      1.2              R    = (R << 1) | B
    12   8424116   23199341.0      2.8      0.8              if R >= D:
    13   4206433   

In [23]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 196.47 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_5288\3149308276.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      5000      34920.0      7.0      0.0      SixCounter    = 0
     5      5000      20895.0      4.2      0.0      SixCounterMax = 0
     6      5000      15779.0      3.2      0.0      j = 0
     7   6957875   18427489.0      2.6      0.9      while N > 0:
     8   6952875   16604115.0      2.4      0.8          try:
     9   6952875   56842157.0      8.2      2.9              Q, R = Cache[j]
    10   6952872   33545428.0      4.8      1.7              R    = (R << 1) | (N & 1)
    11   6952872   18737501.0      2.7      1.0              if R >= D:
    12   3474592   12804653.0      3.7      0.7                  R = R - D
    13   3474

In [55]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 869.917 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_1912\369381277.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      5001      44272.0      8.9      0.0      SixCounter    = 0
     5      5001      19684.0      3.9      0.0      SixCounterMax = 0
     6      5001      17250.0      3.4      0.0      j = 0
     7   6618373   21280476.0      3.2      0.2      while N > 0:
     8                                                   # TODO: Could divide first, which would allow us to remove the (j+1) from the divisor.
     9                                                   # TODO: use continued division for this as well? Or can't we just use the remainder instead of the number?
    10   6613373 6513440895.0    984.9     74.9          Digit = N % 10
    11

In [131]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 6860.08 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_7708\2766715933.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4     49999     331421.0      6.6      0.0      SixCounter    = 0
     5     49999     134709.0      2.7      0.0      SixCounterMax = 0
     6     49999     153801.0      3.1      0.0      j = 0
     7     49999     137143.0      2.7      0.0      N = Number
     8  53829177  150020560.0      2.8      0.2      while N > 0:
     9  53779178        4e+10    740.6     58.1          Digit = N % 10
    10  53779178  148807493.0      2.8      0.2          if Digit == 6:
    11   5385654   16825128.0      3.1      0.0              SixCounter   += 1
    12   5385654   39580574.0      7.3      0.1              SixCounterMax = max(SixCo

In [84]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 22106.9 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_12220\4211618779.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4    195347    1271589.0      6.5      0.0      SixCounter    = 0
     5    195347     629752.0      3.2      0.0      SixCounterMax = 0
     6    195347     448973.0      2.3      0.0      i = 0
     7    195347     554875.0      2.8      0.0      N = Number
     8 192232259  547501112.0      2.8      0.2      while N > 0:
     9 192036912        1e+11    672.5     58.4          Digit = N % 10
    10 192036912  528765814.0      2.8      0.2          if Digit == 6:
    11  19237145   59780868.0      3.1      0.0              SixCounter   += 1
    12  19237145  139016938.0      7.2      0.1              SixCounterMax = max(SixC

In [43]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 94.5 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_12220\4211618779.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4      1000       6186.0      6.2      0.0      SixCounter    = 0
     5      1000       2747.0      2.7      0.0      SixCounterMax = 0
     6      1000       2364.0      2.4      0.0      i = 0
     7      1000       3094.0      3.1      0.0      N = Number
     8    902424    2429788.0      2.7      0.3      while N > 0:
     9    901424  553922105.0    614.5     58.6          Digit = N % 10
    10    901424    2409905.0      2.7      0.3          if Digit == 6:
    11     90467     277185.0      3.1      0.0              SixCounter   += 1
    12     90467     679282.0      7.5      0.1              SixCounterMax = max(SixCoun

In [33]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 95.9172 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_12220\3907487493.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4      1000       7890.0      7.9      0.0      SixCounter    = 0
     5      1000       3153.0      3.2      0.0      SixCounterMax = 0
     6      1000       2561.0      2.6      0.0      i = 0
     7      1000       2442.0      2.4      0.0      N = Number
     8    901749    2333698.0      2.6      0.2      while N > 0:
     9    900749  557088560.0    618.5     58.1          Digit = N % 10
    10    900749    2485078.0      2.8      0.3          if Digit == 6:
    11     90253     288829.0      3.2      0.0              SixCounter   += 1
    12     90253     715613.0      7.9      0.1              SixCounterMax = max(SixC

In [15]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 2697.76 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_1796\3027381636.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(Number, Divisors, Cache):
     4     94909     681102.0      7.2      0.0      SixCounter    = 0
     5     94909     338009.0      3.6      0.0      SixCounterMax = 0
     6     94909     249670.0      2.6      0.0      i = 0
     7     94909     301435.0      3.2      0.0      N = Number
     8  44363833  118510216.0      2.7      0.4      while N > 0:
     9  44268925        1e+10    329.3     54.0          Digit    = N % 10
    10  44268925  665760860.0     15.0      2.5          Divisor  = Divisors[i]
    11  44268925  122325401.0      2.8      0.5          if i in Cache:
    12  44268423  317412938.0      7.2      1.2              Q, R = Cache[i]
    1